In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle



    Only loading the 'en' tokenizer.



# Data

https://www.kaggle.com/c/spooky-author-identification/data

In [2]:
PATH='data/spooky-author/'
%ls {PATH}

models/                test.csv      tmp/       trainData.txt
sample_submission.csv  testData.txt  train.csv


In [3]:
def save_data(df, file_train):
    trainData =""
    for idx, row in df.iterrows():
        data = row['text']
        if trainData == "":
            trainData= data
        else :
            trainData=trainData + " " + data

    file_train.write(trainData)
    file_train.close()
    return trainData

In [4]:
file_train= open(f'{PATH}trainData.txt','w')

In [5]:
df_train = pd.read_csv(f'{PATH}train.csv')

In [6]:
df_train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [7]:
train_data= save_data(df_train, file_train)

In [8]:
df_test = pd.read_csv(f'{PATH}test.csv')

In [9]:
df_test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [10]:
file_test= open(f'{PATH}testData.txt','w')

In [11]:
test_data= save_data(df_test, file_test)

In [12]:
' '.join(spacy_tok(train_data[:100]))

'This process , however , afforded me no means of ascertaining the dimensions of my dungeon ; as I might'

In [13]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [16]:
# same setup as Lesson4-imdb setup
PATH = 'data/spooky-author/'

TRN_PATH =  'trainData.txt'
VAL_PATH = 'testData.txt'
TRN = f'{PATH}trainData.txt'
VAL = f'{PATH}testData.txt'

# Model

In [18]:
# bs = 2 the sentence is too short to learn something meaningful
bs=64; bptt=100

In [19]:
# VAL_PATH is wrong
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)

In [20]:
# missing .from_test_files
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [21]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [22]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(91, 4930, 1, 595187)

In [ ]:
# 'stoi': 'string to int'
TEXT.vocab.stoi['the']

In [ ]:
md.trn_ds[0].text[:10]

torchtext will handle turning this words into integer IDs for us automatically.

In [ ]:
TEXT.numericalize([md.trn_ds[0].text[:10]])

Our `LanguageModelData` object will create batches with 64 columns (that's our batch size), and varying sequence lengths of around 80 tokens (that's our `bptt` parameter - *backprop through time*).

Each batch also contains the exact same data as labels, but one word later in the text - since we're trying to always predict the next word. The labels are flattened into a 1d array.

In [ ]:
next(iter(md.trn_dl))

### Train

We have a number of parameters to set - we'll learn more about these later, but you should find these values suitable for many problems.

In [ ]:
em_sz = 300  # size of each embedding vector 200
nh = 50      # number of hidden activations per layer 500
nl = 3       # number of layers

Researchers have found that large amounts of *momentum* (which we'll learn about later) don't work well with these kinds of *RNN* models, so we create a version of the *Adam* optimizer with less momentum than it's default of `0.9`.

In [ ]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

fastai uses a variant of the state of the art [AWD LSTM Language Model](https://arxiv.org/abs/1708.02182) developed by Stephen Merity. A key feature of this model is that it provides excellent regularization through [Dropout](https://en.wikipedia.org/wiki/Convolutional_neural_network#Dropout). There is no simple way known (yet!) to find the best values of the dropout parameters below - you just have to experiment...

However, the other parameters (`alpha`, `beta`, and `clip`) shouldn't generally need tuning.

In [ ]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

As you can see below, I gradually tuned the language model in a few stages. I possibly could have trained it further (it wasn't yet overfitting), but I didn't have time to experiment more. Maybe you can see if you can train it to a better accuracy! (I used `lr_find` to find a good learning rate, but didn't save the output in this notebook. Feel free to try running it yourself now.)

In [ ]:
#learning rate finder
lrf=learner.lr_find()
learner.sched.plot()

In [ ]:
%time learner.fit(1e-2, n_cycle = 1, wds=1e-6, cycle_len=20, cycle_save_name='spooky_1')

In [ ]:
learner.save_encoder('spooky_2')

In [ ]:
learner.load_encoder('spooky_2')

In [ ]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [ ]:
#learning rate finder
lrf=learner.lr_find()
learner.sched.plot()

In [ ]:
%time learner.fit(3e-3, n_cycle = 1, wds=1e-6, cycle_len=3, cycle_save_name='spooky_3')

In [ ]:
learner.save_encoder('spooky_4')

In [ ]:
learner.load_encoder('spooky_4')

In [ ]:
#learning rate finder
lrf=learner.lr_find()
learner.sched.plot()

In [ ]:
%time learner.fit(1e-4, n_cycle = 1, wds=1e-6, cycle_len=20, cycle_save_name='spooky_5')

In [ ]:
learner.load_cycle('spooky_5', 0)

In the sentiment analysis section, we'll just need half of the language model - the *encoder*, so we save that part.

In [ ]:
learner.save_encoder('spooky_6')

Language modeling accuracy is generally measured using the metric *perplexity*, which is simply `exp()` of the loss function we used.

In [ ]:
math.exp(4.6352)

### Test (Optional)

We can play around with our language model a bit to check it seems to be working OK. First, let's create a short bit of text to 'prime' a set of predictions. We'll use our torchtext field to numericalize it so we can feed it to our language model.

In [ ]:
m=learner.model
ss=""". So, it wasn't quite was I was expecting, but I really liked it anyway! The best"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)
' '.join(s[0])

We haven't yet added methods to make it easy to test a language model, so we'll need to manually go through the steps.

In [ ]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=bs

Let's see what the top 10 predictions were for the next word after our short text:

In [ ]:
nexts = torch.topk(res[-1], 10)[1]
[TEXT.vocab.itos[o] for o in to_np(nexts)]

...and let's see if our model can generate a bit more text all by itself!

In [ ]:
print(ss,"\n")
for i in range(50):
    n=res[-1].topk(2)[1]
    n = n[1] if n.data[0]==0 else n[0]
    print(TEXT.vocab.itos[n.data[0]], end=' ')
    res,*_ = m(n[0].unsqueeze(0))
print('...')

# Predict Authors - WIP

In [ ]:
learner.load_encoder('spooky_4')

In [ ]:
msk = np.random.rand(len(df_train)) < 0.9
df_val = df_train[~msk].reset_index()
df_train = df_train[msk].reset_index()

In [ ]:
df = {'train': df_train, 'val': df_val, 'test': df_test}

In [ ]:
LABEL = data.Field(sequential=False)

In [ ]:
class PredictAuthorDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, dfs, **kwargs):
        fields = [("text", text_field), ("label", label_field)]
        examples = []
        for i in range(dfs[path].values[:,1].shape[0]):
            text = df_train.iloc[i].text
            label = df_train.iloc[i].author
            examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, path,
               train, val, test, dfs, **kwargs):
        return super().splits(path,
            text_field=text_field, label_field=label_field,
            train=train, validation=val, test=test, dfs=dfs, **kwargs)

In [ ]:
splits = PredictAuthorDataset.splits(TEXT, LABEL, '',
                             train='train',
                             val='val', test='test', dfs=df)

In [ ]:
t = splits[1].examples[0]

In [ ]:
t.label, ' '.join(t.text[:10])

In [ ]:
t = splits[1].examples[2]

In [ ]:
t.label, ' '.join(t.text[:10])

In [ ]:
md2 = TextData.from_splits(PATH, splits, 1)

In [ ]:
df_train[:10]

In [ ]:
dropouti, dropout, wdrop, dropoute, dropouth = np.array([0.05,0.05,0.1,0.02,0.05])*6
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropouti=dropouti, dropout=dropout, wdrop=wdrop,
                       dropoute=dropoute, dropouth=dropouth)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)

In [ ]:
# pick the best weight
m3.load_encoder(f'spooky_4')

In [ ]:
m3.clip=25.
lrs=np.array([1e-4,1e-3,1e-2])

In [ ]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])

In [ ]:
log_preds, y = m3.predict_with_targs(is_test=True)

In [ ]:
log_preds[:10],y[:10]

In [ ]:
preds = m3.predict()

In [ ]:
res = np.argmax(log_preds, axis=1)

In [ ]:
res[:100]

In [ ]:
y[:100]

In [ ]:
pre_preds,y = m3.predict_with_targs(True)

In [ ]:
preds =to_np(F.softmax(torch.from_numpy(pre_preds[:,1:])))

In [ ]:
preds.shape, len(splits[2].examples)

In [ ]:
preds[:10]

In [ ]:
df_subm = pd.DataFrame(preds)
df_subm.columns = ['EAP', 'HPL', 'MWS']

In [ ]:
df_subm.shape

In [ ]:
df_subm.head()

In [ ]:
df_test.shape

In [ ]:
df_test.head()

In [ ]:
df_id = df_test.drop('text', axis = 1)

In [ ]:
df_id.head()

In [ ]:
df_result = pd.concat([df_id, df_subm], axis =1 , join ='inner')

In [ ]:
df_result.head()

In [ ]:
df_result.to_csv('data/spooky-author-identification/subm_spooky.csv')

In [ ]:
FileLink('data/spooky-author-identification/subm_spooky.csv')